In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Aspect-based Sentiment Analysis/"

In [3]:
!pip install fairseq
!pip3 install vncorenlp
!pip3 install transformers
!pip install keras-rectified-adam
!pip install fastBPE

     |████████████████████████████████| 307kB 6.0MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2046471 sha256=5fa346bfdcaf7315b98a7993c0dcb9c4a29c37c2a89435e2f2e7da0ca1fa0ca8
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
     |████████████████████████████████| 2.7MB 6.0MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=409794934fa75669ba9706822ca73654afedee93219901823f83bb1ad31d629a
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
     |████████████████████████████████| 1.1MB 5.7MB/s 
     |████████████████████████████████| 3.0MB 20.5MB/s 
     |████████████████████████████████| 1.1MB 54.1MB/s 
     |████████████████████████████████| 890kB 55.7MB/s 
  Created wheel for

In [4]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers

In [5]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2020-10-12 06:12:01--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.86.43, 54.192.86.32, 54.192.86.19, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.86.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  12.9MB/s    in 21s     

2020-10-12 06:12:23 (14.7 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [6]:
phoBERT = 'PhoBERT_base_transformers'

In [7]:
import pickle
import pandas as pd
df = pd.read_csv(path+"VLSP2018/review_train.csv")
df_dev = pd.read_csv(path+'VLSP2018/review_dev.csv')
df_test = pd.read_csv(path+'VLSP2018/review_test.csv')

In [8]:
data_train = df.review.to_list()
label_train = pickle.load(open(path+'VLSP2018/aspect_train.pkl','rb'))
data_dev = df_dev.review.tolist()
label_dev =  pickle.load(open(path+'VLSP2018/aspect_dev.pkl','rb'))
data_test = df_test.review.tolist()
label_test = pickle.load(open(path+'VLSP2018/aspect_test.pkl','rb'))

In [9]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
class BPE():
  bpe_codes = 'PhoBERT_base_transformers/bpe.codes'

args = BPE()
bpe = fastBPE(args)

In [10]:
vocab = Dictionary()
vocab.add_from_file('PhoBERT_base_transformers/dict.txt')

In [11]:
import numpy as np
from tqdm import tqdm
max_sequence_length = 256
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  mask_token = np.zeros((len(lines), max_sequence_length), dtype=np.int32)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
      mask = [0 if i==1 else 1 for i in input_ids]
    outputs[idx,:] = np.array(input_ids)
    mask_token[idx,:] = np.array(mask)
  return outputs, mask_token

In [12]:
X_train, X_train_mask = convert_lines(data_train, vocab, bpe)
X_dev, X_dev_mask = convert_lines(data_dev, vocab, bpe)
X_test, X_test_mask = convert_lines(data_test, vocab, bpe)

100%|██████████| 500/500 [00:00<00:00, 717.72it/s]


In [13]:
y_train = np.array(label_train)
y_dev = np.array(label_dev)
y_test = np.array(label_test)

In [14]:
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D,Bidirectional, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda,MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,LSTM,GRU
from tensorflow.keras.optimizers import Adam

In [15]:
def f1(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return 2 * (K.sum(y_true * y_pred)+ K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

In [18]:
config = RobertaConfig.from_pretrained(
  phoBERT+'/config.json',
  output_hidden_states=True,
  num_labels=12
  )
model_bert = TFRobertaModel.from_pretrained(phoBERT+'/model.bin', config=config, from_pt=True)

input_ids = Input(shape=(256,), name='input_token', dtype='int32')
input_mask = Input(shape=(256,), name='mask_token', dtype='int32')

pretrain = model_bert(input_ids, attention_mask=input_mask)[0]
cls_token = pretrain[:,0,:]
dropout_layer = Dropout(0.2)(cls_token)

dense_layer = Dense(768, activation='relu')(dropout_layer)
dense_layer = Dense(512, activation='relu')(dense_layer)
dense_layer = Dense(256, activation='relu')(dense_layer)
preds = Dense(12, activation='sigmoid')(dense_layer)

model = Model([input_ids,input_mask], preds)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [19]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights
cw = calculating_class_weights(y_train)

In [20]:
LR = 3e-5
EPOCHS = 10
BATCH_SIZE = 16

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=LR),
              metrics=[f1])

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint 
checkpoint = ModelCheckpoint(path+'phoBERT_checkpoint', monitor='val_f1', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 169s 909ms/step - loss: 0.3891 - f1: 0.5286 - val_loss: 0.5014 - val_f1: 0.6073
Epoch 2/10
186/186 [==============================] - 167s 897ms/step - loss: 0.2923 - f1: 0.6418 - val_loss: 0.4268 - val_f1: 0.6566
Epoch 3/10
186/186 [==============================] - 167s 896ms/step - loss: 0.2291 - f1: 0.7181 - val_loss: 0.3747 - val_f1: 0.6933
Epoch 4/10
186/186 [==============================] - 167s 896ms/step - loss: 0.1842 - f1: 0.7743 - val_loss: 0.3732 - val_f1: 0.7320
Epoch 5/10
186/186 [==============================] - 167s 896ms/step - loss: 0.1557 - f1: 0.8112 - val_loss: 0.3963 - val_f1: 0.7483
Epoch 6/10
186/186 [==============================] - 167s 898ms/step - loss: 0.1223 - f1: 0.8493 - val_loss: 0.4297 - val_f1: 0.7273
Epoch 7/10
186/186 [==============================] - 167s 896ms/step - loss: 0.0972 - f1: 0.8795 - val_loss: 0.4868 - val_f1: 0.7543
Epoch 8/10
186/186 [==============================] - 167s 896

In [ ]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 99s 530ms/step - loss: 0.0580 - f1: 0.9255 - val_loss: 0.5174 - val_f1: 0.7658
Epoch 2/10
186/186 [==============================] - 98s 529ms/step - loss: 0.0474 - f1: 0.9386 - val_loss: 0.5312 - val_f1: 0.7783
Epoch 3/10
186/186 [==============================] - 98s 530ms/step - loss: 0.0454 - f1: 0.9429 - val_loss: 0.6108 - val_f1: 0.7782
Epoch 4/10
186/186 [==============================] - 98s 529ms/step - loss: 0.0330 - f1: 0.9569 - val_loss: 0.5881 - val_f1: 0.7867
Epoch 5/10
186/186 [==============================] - 98s 529ms/step - loss: 0.0244 - f1: 0.9669 - val_loss: 0.6549 - val_f1: 0.7821
Epoch 6/10
186/186 [==============================] - 98s 528ms/step - loss: 0.0239 - f1: 0.9693 - val_loss: 0.6737 - val_f1: 0.7889
Epoch 7/10
186/186 [==============================] - 98s 529ms/step - loss: 0.0262 - f1: 0.9678 - val_loss: 0.6857 - val_f1: 0.7868
Epoch 8/10
186/186 [==============================] - 98s 528ms/step 

In [ ]:

EPOCHS = 20
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 4
LR = 0.0001
LR_DC_STEP = 80 
LR_DC = 0.1
CUR_DIR = os.path.dirname(os.getcwd())
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4
CKPT_PATH2 = 'model_ckpt2'

if not os.path.exists(CKPT_PATH2):
    os.mkdir(CKPT_PATH2)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train,dtype=torch.long), torch.tensor(y_train,dtype=torch.long))
valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X_dev,dtype=torch.long), torch.tensor(y_dev,dtype=torch.long))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
param_optimizer = list(model_bert.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

num_train_optimization_steps = int(EPOCHS*len(train_dataset)/BATCH_SIZE/ACCUMULATION_STEPS)
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # scheduler với linear warmup
scheduler0 = get_constant_schedule(optimizer)  # scheduler với hằng số
# optimizer = optim.Adam(phoBERT_cls.parameters(), LR)
criteria = nn.BCEWithLogitsLoss()
# scheduler = StepLR(optimizer, step_size = LR_DC_STEP, gamma = LR_DC)
avg_loss = 0.
avg_accuracy = 0.
frozen = True

In [ ]:
label_dev = np.array(label_dev)

In [ ]:
frozen = True
for epoch in tqdm(range(EPOCHS)):
  if epoch > 0 and frozen:
      for child in tsfm.children():
          for param in child.parameters():
              param.requires_grad = True
      frozen = False
      del scheduler0
      torch.cuda.empty_cache()

  val_preds = None
  avg_loss = 0.
  avg_accuracy = 0.

  for i,(x_batch, y_batch) in enumerate(train_loader):
      model_bert.train()
      
      y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
      loss =  criteria(y_pred.cuda(),y_batch.float().cuda())
      loss.backward()
      optimizer.step()
      if not frozen:
          scheduler.step()
      else:
          scheduler0.step()
      optimizer.zero_grad()
      avg_loss = loss.item() / len(train_loader)
  print('avg_loss: ',avg_loss)
  model_bert.eval()
  pred_labels, true_labels = [], []
  for j,(x_batch_valid, y_batch_valid) in enumerate(valid_loader):
    y_pred_valid = model_bert(x_batch_valid.cuda(), attention_mask=(x_batch_valid>0).cuda())
    y_pred_valid = y_pred_valid.squeeze().detach().cpu().numpy()
    pred_labels.append(y_pred_valid)
    true_labels.append(y_batch_valid.squeeze().detach().cpu().numpy())

  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  score = f1_score(true_bools, pred_bools, average ='micro')
  print(f"F1 score @0.5 = {score:.4f}")








  0%|          | 0/20 [00:00<?, ?it/s]

avg_loss:  0.000825245129434686









  5%|▌         | 1/20 [01:39<31:37, 99.86s/it]

F1 score @0.5 = 0.5565
avg_loss:  0.0009285634466511036









 10%|█         | 2/20 [03:19<29:57, 99.85s/it]

F1 score @0.5 = 0.4902
avg_loss:  0.0007592557292235525









 15%|█▌        | 3/20 [04:59<28:17, 99.85s/it]

F1 score @0.5 = 0.4902


In [ ]:
y_pred = model_bert(x_batch_valid.cuda(), attention_mask=(x_batch_valid>0).cuda())

y_pred = y_pred.detach().cpu().numpy()

In [ ]:
a = [[1,2,3,4]]
b = [[5,5,67,7]]
a +=b
a = np.array(a)
a>5

array([[False, False, False, False],
       [False, False,  True,  True]])

In [ ]:
y_pred>0.5

array([[False, False, False,  True,  True, False, False, False, False,
        False, False, False],
       [False, False, False,  True,  True, False, False, False, False,
        False, False, False],
       [False, False, False,  True,  True, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True,  True, False, False, False, False,
        False, False, False]])

In [ ]:
k

array([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0]])

In [ ]:
f1_score(y_pred>0.5, k, average='samples')

ValueError: ignored